Construcción de BD para el primer ejercicio

In [3]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE empleados (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    departamento TEXT,
    salario INTEGER,
    fecha_contratacion TEXT
)
""")

empleados_data = [
    (1, 'Ana', 'Ventas', 5000, '2020-01-15'),
    (2, 'Luis', 'IT', 6000, '2019-08-21'),
    (3, 'María', 'Ventas', 5500, '2021-05-10'),
    (4, 'Juan', 'IT', 7000, '2018-11-03')
]

cursor.executemany("INSERT INTO empleados VALUES (?, ?, ?, ?, ?)", empleados_data)
conn.commit()

pd.read_sql("SELECT * FROM empleados", conn)


,id,nombre,departamento,salario,fecha_contratacion
0,1,Ana,Ventas,5000,2020-01-15
1,2,Luis,IT,6000,2019-08-21
2,3,María,Ventas,5500,2021-05-10
3,4,Juan,IT,7000,2018-11-03


1. Escriba una consulta SQL para obtener el promedio de salario por departamento, excluyendo empleados contratados después del 1 de enero de 2020.

In [5]:
pd.read_sql("""
SELECT 
    departamento,
    AVG(salario) AS promedio_salario
FROM empleados
WHERE fecha_contratacion <= '2020-01-01'
GROUP BY departamento;
""", conn)


,departamento,promedio_salario
0,IT,6500.0


Creación de tablas con datos aleatorios (Ventas y Clientes) para los demás ejercicios

In [6]:
cursor.execute("DROP TABLE IF EXISTS ventas;")
cursor.execute("DROP TABLE IF EXISTS clientes;")

# tabla clientes
cursor.execute("""
CREATE TABLE clientes (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    apellido TEXT
);
""")

# tabla ventas
cursor.execute("""
CREATE TABLE ventas (
    id INTEGER PRIMARY KEY,
    cliente_id INTEGER,
    producto TEXT,
    fecha DATE,
    monto DECIMAL(10,2),
    FOREIGN KEY (cliente_id) REFERENCES clientes(id)
);
""")

# Datos Aleatorios

clientes_data = [
    (1, 'Ana', 'Ramírez'),
    (2, 'Luis', 'Gómez'),
    (3, 'María', 'Pérez'),
    (4, 'Juan', 'Torres'),
    (5, 'Camila', 'López'),
    (6, 'Andrés', 'Castro'),
    (7, 'Valentina', 'Ruiz'),
    (8, 'Carlos', 'Martínez'),
    (9, 'Laura', 'Sánchez'),
    (10, 'Jorge', 'Morales')
]
cursor.executemany("INSERT INTO clientes VALUES (?, ?, ?)", clientes_data)

ventas_data = [
    (1, 1, 'Celular', '2025-02-15', 850.00),
    (2, 1, 'Audífonos', '2025-03-05', 120.00),
    (3, 2, 'Laptop', '2025-04-12', 1500.00),
    (4, 3, 'Tablet', '2025-05-10', 900.00),
    (5, 4, 'Smartwatch', '2025-07-20', 400.00),
    (6, 4, 'Celular', '2025-09-01', 950.00),
    (7, 5, 'Laptop', '2025-06-25', 1400.00),
    (8, 6, 'Audífonos', '2025-08-15', 130.00),
    (9, 6, 'Smartwatch', '2025-09-28', 350.00),
    (10, 7, 'Tablet', '2025-01-18', 750.00),
    (11, 7, 'Celular', '2025-09-30', 890.00),
    (12, 8, 'Laptop', '2025-03-20', 1600.00),
    (13, 9, 'Celular', '2025-05-11', 870.00),
    (14, 9, 'Audífonos', '2025-09-09', 110.00),
    (15, 10, 'Smartwatch', '2025-10-01', 420.00)
]
cursor.executemany("INSERT INTO ventas VALUES (?, ?, ?, ?, ?)", ventas_data)
conn.commit()

print("Tabla clientes:")
for row in cursor.execute("SELECT * FROM clientes;"):
    print(row)

print("\nTabla ventas:")
for row in cursor.execute("SELECT * FROM ventas;"):
    print(row)


Tabla clientes:
(1, 'Ana', 'Ramírez')
(2, 'Luis', 'Gómez')
(3, 'María', 'Pérez')
(4, 'Juan', 'Torres')
(5, 'Camila', 'López')
(6, 'Andrés', 'Castro')
(7, 'Valentina', 'Ruiz')
(8, 'Carlos', 'Martínez')
(9, 'Laura', 'Sánchez')
(10, 'Jorge', 'Morales')

Tabla ventas:
(1, 1, 'Celular', '2025-02-15', 850)
(2, 1, 'Audífonos', '2025-03-05', 120)
(3, 2, 'Laptop', '2025-04-12', 1500)
(4, 3, 'Tablet', '2025-05-10', 900)
(5, 4, 'Smartwatch', '2025-07-20', 400)
(6, 4, 'Celular', '2025-09-01', 950)
(7, 5, 'Laptop', '2025-06-25', 1400)
(8, 6, 'Audífonos', '2025-08-15', 130)
(9, 6, 'Smartwatch', '2025-09-28', 350)
(10, 7, 'Tablet', '2025-01-18', 750)
(11, 7, 'Celular', '2025-09-30', 890)
(12, 8, 'Laptop', '2025-03-20', 1600)
(13, 9, 'Celular', '2025-05-11', 870)
(14, 9, 'Audífonos', '2025-09-09', 110)
(15, 10, 'Smartwatch', '2025-10-01', 420)


2. Escribe una consulta para obtener los 5 clientes con mayor monto total de ventas en los últimos 6 meses. 

In [7]:
pd.read_sql("""
SELECT 
    c.nombre,
    c.apellido,
    SUM(v.monto) AS total_ventas
FROM ventas v
JOIN clientes c ON v.cliente_id = c.id
WHERE v.fecha >= DATE('now', '-6 months')
GROUP BY c.id, c.nombre, c.apellido
ORDER BY total_ventas DESC
LIMIT 5;
""", conn)

,nombre,apellido,total_ventas
0,Camila,López,1400
1,Juan,Torres,1350
2,Laura,Sánchez,980
3,María,Pérez,900
4,Valentina,Ruiz,890


3. Escribe una consulta para calcular el ticket promedio de ventas por cliente en el último año. 

In [8]:
pd.read_sql("""
SELECT 
    c.nombre,
    c.apellido,
    ROUND(AVG(v.monto), 2) AS ticket_promedio
FROM ventas v
JOIN clientes c ON v.cliente_id = c.id
WHERE v.fecha >= DATE('now', '-1 year')
GROUP BY c.id, c.nombre, c.apellido
ORDER BY ticket_promedio DESC;
""", conn)


,nombre,apellido,ticket_promedio
0,Carlos,Martínez,1600.0
1,Luis,Gómez,1500.0
2,Camila,López,1400.0
3,María,Pérez,900.0
4,Valentina,Ruiz,820.0
5,Juan,Torres,675.0
6,Laura,Sánchez,490.0
7,Ana,Ramírez,485.0
8,Jorge,Morales,420.0
9,Andrés,Castro,240.0


4. Escribe una consulta para obtener el nombre completo de los clientes y su monto total de ventas.

In [9]:
pd.read_sql("""
SELECT 
    c.nombre || ' ' || c.apellido AS nombre_completo,
    ROUND(SUM(v.monto), 2) AS monto_total
FROM ventas v
JOIN clientes c ON v.cliente_id = c.id
GROUP BY c.id, c.nombre, c.apellido
ORDER BY monto_total DESC;
""", conn)

,nombre_completo,monto_total
0,Valentina Ruiz,1640.0
1,Carlos Martínez,1600.0
2,Luis Gómez,1500.0
3,Camila López,1400.0
4,Juan Torres,1350.0
5,Laura Sánchez,980.0
6,Ana Ramírez,970.0
7,María Pérez,900.0
8,Andrés Castro,480.0
9,Jorge Morales,420.0


5. Escribe una consulta para obtener el ingreso promedio de ventas por mes.

In [10]:
pd.read_sql("""
SELECT 
    ROUND(AVG(ingreso_mensual), 2) AS ingreso_promedio_mensual
FROM (
    SELECT 
        strftime('%Y-%m', fecha) AS anio_mes,
        SUM(monto) AS ingreso_mensual
    FROM ventas
    GROUP BY anio_mes
);
""", conn)


,ingreso_promedio_mensual
0,1124.0


6. Escribe una consulta para calcular el ranking de clientes por ventas en el último año.

In [11]:
pd.read_sql("""
SELECT 
    c.nombre || ' ' || c.apellido AS nombre_completo,
    ROUND(SUM(v.monto), 2) AS total_ventas,
    RANK() OVER (ORDER BY SUM(v.monto) DESC) AS ranking
FROM ventas v
JOIN clientes c ON v.cliente_id = c.id
WHERE v.fecha >= DATE('now', '-1 year')
GROUP BY c.id, c.nombre, c.apellido
ORDER BY ranking;
""", conn)


,nombre_completo,total_ventas,ranking
0,Valentina Ruiz,1640.0,1
1,Carlos Martínez,1600.0,2
2,Luis Gómez,1500.0,3
3,Camila López,1400.0,4
4,Juan Torres,1350.0,5
5,Laura Sánchez,980.0,6
6,Ana Ramírez,970.0,7
7,María Pérez,900.0,8
8,Andrés Castro,480.0,9
9,Jorge Morales,420.0,10


7. Escribe una consulta para calcular el total de ventas por cliente y luego selecciona solo los clientes cuyo total de ventas es superior al promedio general.

In [13]:
pd.read_sql("""
WITH ventas_por_cliente AS (
    SELECT 
        c.id,
        c.nombre || ' ' || c.apellido AS nombre_completo,
        SUM(v.monto) AS total_ventas
    FROM ventas v
    JOIN clientes c ON v.cliente_id = c.id
    GROUP BY c.id, c.nombre, c.apellido
),
promedio_general AS (
    SELECT AVG(total_ventas) AS promedio FROM ventas_por_cliente
)
SELECT 
    vpc.nombre_completo,
    ROUND(vpc.total_ventas, 2) AS total_ventas
FROM ventas_por_cliente vpc, promedio_general pg
WHERE vpc.total_ventas > pg.promedio
ORDER BY vpc.total_ventas DESC;
""", conn)


,nombre_completo,total_ventas
0,Valentina Ruiz,1640.0
1,Carlos Martínez,1600.0
2,Luis Gómez,1500.0
3,Camila López,1400.0
4,Juan Torres,1350.0
